In [257]:
import operator
import copy

operator_dict = {
    "<" : operator.lt,
    "=" : operator.eq,
    ">" : operator.gt,
}

op_list = ["<" , "=" , ">"]

type_dict = {
    "int" : int,
    "str" : str,
    "string" : str,
    "float" : float,
}

class Database:
    
    def __init__(self,filename):
        
        self.colType = []
        self.colName = []
        sigma = []
        self.data = []
        self.readfile(filename)
        
    def readfile(self,filename):
        #Read file data.txt
        file = open(filename,"r")
        data = file.readlines()
        self.colName = data[0]
        del data[0]
        self.colType = data[0]
        self.colType = self.colType.split()
        del data[0]
        # now data is a list of list: each list in data is a row of the database
        for i in range (len(data)):
            data[i] =  data[i].split()
        self.data = data
        
            
    def add_DC(self,dc):
        self.sigma.append(dc)
    
    def is_satisfy_by(self, Sigma):
        for dc in Sigma:
            if not(len(dc.get_violation)==0) :
                return False
        return True
    def is_satisfy(self):
        return is_satisfy_by(self.Sigma)
    #Todo 
    #def write(filename):
    
    #def Theta-tolerant(thetha):
    
    #def RepairData(thetha):

class Predicate:
    #Predicates contain an attribute (here a position so a number)
    #and an operator. See operator_dict for all possible operator.
    def __init__(self, attributePos, operator, attributeType):
        self.attribute = attributePos #int
        self.operator = operator #list of string
        self.attributeType = attributeType
    #return true if (row1,row2) does satisfy the predicates
    def is_true(self,row1,row2):
        for op in self.operator:
            if operator_dict[op](type_dict[self.attributeType](row1[self.attribute]),type_dict[self.attributeType](row2[self.attribute])):
                return True
        return False
    def __str__(self):
        return "(" +str(self.__dict__) + ")"

    def __eq__(self, other): 
        return self.__dict__ == other.__dict__
    
    def inverse_predicate(self):
        op2 = []
        for elem in op_list:
            if elem not in self.operator:
                op2.append(elem)
        self.operator = op2
        
    def partiesliste(self,seq):
        p = []
        i, imax = 0, 2**len(seq)-1
        while i <= imax:
            s = []
            j, jmax = 0, len(seq)-1
            while j <= jmax:
                if (i>>j)&1 == 1:
                    s.append(seq[j])
                j += 1
            p.append(s)
            i += 1 
        return p    
        
    def get_weaker(self):
        weaker_pred = []
        weak_op = self.partiesliste(self.operator)
        for elem in weak_op:
            weaker_pred.append(Predicate(self.attribute,elem,self.attributeType))
        return weaker_pred
    
    def get_weaker_not_empty(self):
        weak = self.get_weaker()
        returnList = []
        for i in range(len(weak)):
            if not(len(weak[i].operator) == 0):
                returnList.append(weak[i])
        return returnList
    
class DC:
     #A denial constraint is a conjunction of multiple predicates.
    #They cannot be all true at the same time
    
    #predicates should be none or a list of predicates!
    def __init__(self,relSize,colType):
        self.predicates = []
        for i in range(relSize):
            self.add(Predicate(i,["<","=",">"],colType[i]))
    
    def add(self,pred):
        self.predicates.append(pred)
        
    def change_predicate(self,predic):
        for pred in self.predicates:
            if pred.attribute == predic.attribute:
                pred.operator = predic.operator
                return None;
        
    def __str__(self):
        stringreturn = "{"
        for pred in self.predicates:
            stringreturn = stringreturn + str(pred) + ","
        return stringreturn + "}"
    
    def remove(self,predicate):
        for pred in self.predicates:
            if pred==predicate:
                del pred
    
    def size(self):
        return len(self.predicates)
    
    #return true if at least one of the predicates is false    
    def is_satisfy(self,row1,row2):
        for predicate in self.predicates:
            if not(predicate.is_true(row1,row2)):
                return True
        return False
    
    def get_violations(self,data):
        viol_set =[]
        print("start")
        for i in range(len(data)):
            for j in range (len(data)):
                if (i != j and not(self.is_satisfy(data[i],data[j]))):
                    tupleviol = (i,j)         
                    viol_set.append(tupleviol)            
        return viol_set
    
    def is_satisfy_by_data(self,data):
        #data non vide
        if data == None or len(data)==0:
            return False
        for row1 in data:
            for row2 in data:
                if not(self.is_satisfy(row1,row2)):
                    return False
        return True
    # return true if self is a logical implication of DC
    # i.e self implies (logicaly) DC
    # uses Lemma 2
    def implies(self,DC):
        for i in range(len(self.predicates)):
            pred1 = self.predicates[i]
            pred2 = DC.predicates[i]
            if not(pred2.operator <= pred1.operator):
                return False
        return True
    
    def is_trivial(self):
        for pred in self.predicates:
            if len(pred.operator) != 3:
                return False
        return True
    
    #satisfiable : see definition section 2.3.1.1
    def satisfiable(self):
        for pred in self.predicates:
            if "=" not in pred.operator:
                return True
        return False
            
    def get_variations(self):
        var = self.get_var_recur(0,self)
        var_weak = []
       # for elem in var:
       #     if not(elem.is_trivial()) and elem.satisfiable() and elem.is_weaker(self):
       #         var_weak.append(elem)
       # return var_weak
        return var
    
    def get_var_recur(self,i,dc):
        returnList = []
        if(i<len(dc.predicates)):
            weak_pred = self.predicates[i].get_weaker_not_empty()
            for elem in weak_pred:
                new_dc = copy.deepcopy(dc)
                new_dc.predicates[i] = elem
                returnList.append(new_dc)
                returnList = returnList + self.get_var_recur(i+1,new_dc)
        return returnList

        
    def is_weaker(self,DC):
        return DC.implies(self)
    
db3 = Database("datasize1.txt")
data3 = db3.data
den = DC(len(db3.colType),db3.colType)
print(den.size())
var = den.get_variations()
print('yohhou')
print(len(var))
for elem in var:
    print(elem)

1
yohhou
7
{({'attribute': 0, 'operator': ['<'], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['='], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['<', '='], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['>'], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['<', '>'], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['=', '>'], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['<', '=', '>'], 'attributeType': 'int'}),}


In [244]:
#Launch a Database
db = Database("Data.txt")
data = db.data
#Create some DC
#DC1
#create predicates
pred1 = Predicate(4,[">"],db.colType[4])
print(pred1.is_true(data[0],data[1])==False)
print(pred1.is_true(data[1],data[0])==True)
pred2 = Predicate(5,["<"],db.colType[5])
print(pred2.is_true(data[0],data[1])==False)
print(pred2.is_true(data[1],data[0])==False)

den1 = DC(len(db.colType), db.colType)
den1.change_predicate(pred1)
print(den1.is_satisfy(data[0],data[1]) == True) 
print(den1.is_satisfy(data[1],data[0]) == False) 
print(den1.size())
den1.change_predicate(pred2)
print(den1.is_satisfy(data[0],data[1])==True) 
print(den1.is_satisfy(data[1],data[0])==True)
print(den1.size())
print(den1.is_satisfy(data[4],data[3])==False) 

print("____")
print(pred1.is_true(data[4],data[3]))
print(pred2.is_true(data[4],data[3]))
print("____")
print(den1.get_violations(data) == [(4, 3), (5, 3), (6, 3)])

True
True
True
True
True
True
6
True
True
6
True
____
True
True
____
start
True


In [188]:
#test trivialité

den2 = DC(len(db.colType), db.colType)
print(den2.is_trivial()==True)
#test implies
pred3 = Predicate(5,["<",">"],db.colType[5])
den2.change_predicate(pred1)
den2.change_predicate(pred3)
print(den2.is_satisfy_by_data(data)==False)
print(den1.implies(den2) ==False)
print(den2.implies(den1) ==True)
print(den2.is_trivial()==False)

True
True
True
True
True


In [189]:
a = pred3.get_weaker()
for elem in a:
    print(elem)
print(den2.satisfiable())
pred4 = Predicate(2,["<","=",">"],db.colType[2])
b = pred4.get_weaker()
for elem in b:
    print(elem)

({'attribute': 5, 'operator': [], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['<'], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['>'], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['<', '>'], 'attributeType': 'int'})
True
({'attribute': 2, 'operator': [], 'attributeType': 'string'})
({'attribute': 2, 'operator': ['<'], 'attributeType': 'string'})
({'attribute': 2, 'operator': ['='], 'attributeType': 'string'})
({'attribute': 2, 'operator': ['<', '='], 'attributeType': 'string'})
({'attribute': 2, 'operator': ['>'], 'attributeType': 'string'})
({'attribute': 2, 'operator': ['<', '>'], 'attributeType': 'string'})
({'attribute': 2, 'operator': ['=', '>'], 'attributeType': 'string'})
({'attribute': 2, 'operator': ['<', '=', '>'], 'attributeType': 'string'})


In [190]:
var = den2.get_variations()
for elem in var:
    for elem2 in elem.predicates:
        print(elem2)
print(var)
print(len(var))

8
8
8
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2


4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
8
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2


4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
8
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4
8
2
4
4
2
4
4
2
4
4
2
4
4
2
4
4


In [255]:
#Launch a Database
db2 = Database("testData.txt")
data2 = db2.data
predicate1 = Predicate(0,["<"],db2.colType[0])
predicate2 = Predicate(1,[">"],db2.colType[1])
den3 =  DC(len(db2.colType),db2.colType)
den3.change_predicate(predicate1)
den3.change_predicate(predicate2)
print(den3.is_satisfy(data2[2],data2[1])) 
print(den3.is_satisfy(data2[1],data2[2]))
pred1var = predicate1.get_weaker()
for elem in pred1var:
    print(elem)
pred1var = predicate1.get_weaker_not_empty()
print("yohou")
for elem in pred1var:
    print(elem)
var =  den3.get_variations()
print("yolo")
for elem in var:
    print(elem)
print(len(var))

True
True
({'attribute': 0, 'operator': [], 'attributeType': 'int'})
({'attribute': 0, 'operator': ['<'], 'attributeType': 'int'})
yohou
({'attribute': 0, 'operator': ['<'], 'attributeType': 'int'})
yolo
{({'attribute': 0, 'operator': ['<'], 'attributeType': 'int'}),({'attribute': 1, 'operator': ['>'], 'attributeType': 'int'}),({'attribute': 2, 'operator': ['<', '=', '>'], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['<'], 'attributeType': 'int'}),({'attribute': 1, 'operator': ['>'], 'attributeType': 'int'}),({'attribute': 2, 'operator': ['<', '=', '>'], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['<'], 'attributeType': 'int'}),({'attribute': 1, 'operator': ['>'], 'attributeType': 'int'}),({'attribute': 2, 'operator': ['<'], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['<'], 'attributeType': 'int'}),({'attribute': 1, 'operator': ['>'], 'attributeType': 'int'}),({'attribute': 2, 'operator': ['='], 'attributeType': 'int'}),}
{({'attribute': 0, '

[1, 4]
[1, 4]
